In [1]:
import matplotlib.pyplot as plt
import numpy as np

import gpflow
import pandas as pd
import os
import matplotlib.pyplot as plt
import re
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import time
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn import preprocessing
from xgboost import XGBRegressor
import joblib

2024-06-16 13:30:48.452734: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-16 13:30:49.485078: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /share/apps/22.04/openmpi/4.1.5/lib:/share/apps/22.04/slurm/23.02.7/lib
2024-06-16 13:30:49.485144: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-06-16 13:30:49.651620: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has al

In [2]:
pfts = ["Tropical", "Temperate_Broadleaf","Temperate_Deciduous"
        ,"Temperate_Coniferous","Boreal"]

var_list = ['GPP', 'NPP', 'VEGC', 'SOILORGC']


In [3]:
pft = pfts[0]
all_model_data = pd.read_csv('/home/smmrrr/TEM_Analysis/TEM_Analysis/model_data/merged_outliers_removed/'+pft+'_cru_hist_canesm5_ssp245.csv')


In [57]:
### get 15% hold out
pred_vars = ['ordinal_stand_age'
                         ,'nirr_avg', 'nirr_min', 'nirr_max', 'prec_avg',
       'prec_min', 'prec_max', 'tair_avg', 'tair_min', 'tair_max',
       'trange_avg', 'trange_min', 'trange_max', 'vpr_avg',
       'vpr_min', 'vpr_max', 'wind_avg', 'wind_min', 'wind_max','s1', 's2', 'co2']
# X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.15, random_state=42)
# Get the indices and split them
X = np.array(all_model_data[pred_vars])
Y = np.array(all_model_data[var])

# Split the data into training and testing sets (80% train, 20% test)
# randomness can be controlled with a defined `random_state`
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.99, random_state=42)
# Y = np.array(
#     model_data.loc[:, 'monthly_mean_ratio']
# )
# X = np.array(
#      # model_data.loc[model_data['year']>=2020,['ordinal_stand_age']]
#      model_data.loc[:,['ordinal_stand_age','tmp_ratio']]
# )
Y = Y.reshape(len(Y),1)
Y_train = Y_train.reshape(len(Y_train),1)
Y_test = Y_test.reshape(len(Y_test),1)



In [58]:
print(len(X_test), ' ',len(X_train))

13531265   136679


In [10]:
410038/13257906

0.03092781016851379

In [12]:
var = var_list[0]

In [36]:
Y_train = Y_train.reshape(len(Y_train),1)
Y_test = Y_test.reshape(len(Y_test),1)


In [59]:
percentiles = [0, 10, 25, 50, 75, 90, 100]
percentile_values = np.percentile(Y_train, percentiles)
print(percentile_values)

[  0.     36.678 112.805 158.25  193.93  226.752 324.7  ]


In [60]:
rng = np.random.default_rng(12345)
bin_size = 40

# Calculate the percentile values on Y
percentiles = [0, 10, 25, 50, 75, 90, 100]
percentile_values = np.percentile(Y_train, percentiles)
print(percentile_values)

# Initialize a list to store the indices of the subsets
subset_indices = []

# For each pair of percentile values, select the indices of `bin_size` points from within that range
for i in range(len(percentile_values) - 1):
    # Get the value range for the current bin
    low, high = percentile_values[i], percentile_values[i+1]

    # Find the indices where Y falls within the current percentile range
    in_range_indices = np.where((Y_train >= low) & (Y_train < high))[0]
    print(in_range_indices)

    # If there are fewer than bin_size indices, take all of them
    if len(in_range_indices) < bin_size:
        selected_indices = in_range_indices
    else:
        # Otherwise, randomly select `bin_size` indices within this range
        selected_indices = rng.choice(in_range_indices, size=bin_size, replace=False)

    # Extend the subset_indices list with the selected indices
    subset_indices.extend(selected_indices)

# Convert the list of indices to a numpy array and reshape if necessary
subset_indices = np.array(subset_indices).reshape(-1, 1)
print("Shape of subset_indices:", subset_indices.shape)

# Using the selected indices, we can now reference the corresponding rows from X
inducing_variable = X_train[subset_indices.flatten(), :]
print("Shape of inducing_variable:", inducing_variable.shape)
inducing_variable = X_train[subset_indices.flatten(),:]

[  0.     36.678 112.805 158.25  193.93  226.752 324.7  ]
[     4     10     12 ... 136657 136673 136676]
[     0      5     15 ... 136641 136662 136663]
[     1      6      8 ... 136659 136666 136670]
[     7     13     14 ... 136675 136677 136678]
[    18     19     53 ... 136651 136671 136672]
[     2      3     16 ... 136655 136658 136668]
Shape of subset_indices: (240, 1)
Shape of inducing_variable: (240, 22)


In [51]:
inducing_variable.shape

array([[  1.  , 229.16, 184.32, ...,  33.94,  24.03, 301.76],
       [  3.  , 265.77, 210.31, ...,  67.62,  19.16, 575.47],
       [  5.  , 256.41, 221.03, ...,  49.04,  18.09, 299.84],
       ...,
       [ 22.  , 196.81, 143.49, ...,  39.26,  20.28, 579.54],
       [  6.  , 264.15, 167.78, ...,  44.63,  30.32, 588.38],
       [  5.  , 212.68, 173.08, ...,  45.69,  21.81, 406.94]])

In [46]:
np.array(np.where((Y_train >= low) & (Y_train < high))).shape 


(2, 40996)

In [45]:
Y_train.shape

(410038, 1)

In [40]:
subset_indices

array([], dtype=float64)

In [33]:
X_train.shape

(410038, 22)

In [61]:
model = gpflow.models.SGPR(
    (X_train, Y_train),
    kernel=gpflow.kernels.RBF(),
    # kernel=gpflow.kernels.SquaredExponential(),
    inducing_variable=inducing_variable,
        # noise_variance=1.0  # Set to some reasonable initial value
)

In [53]:
print(gpflow.utilities.print_summary(model, "notebook"))

name,class,transform,prior,trainable,shape,dtype,value
SGPR.kernel.variance,Parameter,Softplus,,True,(),float64,1.0
SGPR.kernel.lengthscales,Parameter,Softplus,,True,(),float64,1.0
SGPR.likelihood.variance,Parameter,Softplus + Shift,,True,(),float64,1.0
SGPR.inducing_variable.Z,Parameter,Identity,,True,"(240, 22)",float64,"[[1., 229.16, 184.32..."


None


In [ ]:
import tensorflow as tf

start_time = time.time()
# Instantiate an Adam optimizer with a set learning rate
adam_optimizer = tf.optimizers.Adam(learning_rate=0.01)

# Run the optimization loop
# Perform a number of optimization steps
num_iterations = 1000  # Choose an appropriate number of iterations for your problem
for i in range(num_iterations):
    adam_optimizer.minimize(model.training_loss, model.trainable_variables)
    if (i % 100 ==0):
        print(i)
        elapsed_time_minutes = (time.time() - start_time) / 60
        print(f"Elapsed time: {elapsed_time_minutes:.2f} minutes")

0
Elapsed time: 0.16 minutes


In [56]:
# Mini-batch training can be handled by SVGP in GPflow, which allows online training with mini-batches
model = gpflow.models.SVGP(
    kernel=gpflow.kernels.RBF(),
    likelihood=gpflow.likelihoods.Gaussian(),
    inducing_variable=inducing_variable,
    num_data=X_train.shape[0]  # Provide the total number of data points for scaling
)
start_time = time.time()

# Instantiate an Adam optimizer with a set learning rate
initial_learning_rate = 0.1
adam_optimizer = tf.optimizers.Adam(learning_rate=initial_learning_rate)

# Perform optimization using the Adam optimizer
num_iterations = 1000
batch_size = 1000

# Data pipeline for mini-batch training
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, Y_train))
train_dataset = train_dataset.shuffle(buffer_size=len(X_train)).batch(batch_size)

# Optimization loop
for i in range(num_iterations):
    for (X_batch, Y_batch) in train_dataset:
        with tf.GradientTape() as tape:
            loss = model.training_loss((X_batch, Y_batch))
        gradients = tape.gradient(loss, model.trainable_variables)
        adam_optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    # Log progress every 100 iterations
    if i % 100 == 0:
        elapsed_time_minutes = (time.time() - start_time) / 60
        print(f"Iteration {i}, Elapsed time: {elapsed_time_minutes:.2f} minutes")

Iteration 0, Elapsed time: 1.08 minutes


KeyboardInterrupt: 

In [ ]:
print(gpflow.utilities.print_summary(model, "notebook"))
###save model
model.compiled_predict_f = tf.function(
    lambda Xnew: model.predict_f(Xnew, full_cov=False),
    input_signature=[tf.TensorSpec(shape=[None, X_train.shape[1]], dtype=tf.float64)],
)
model.compiled_predict_y = tf.function(
    lambda Xnew: model.predict_y(Xnew, full_cov=False),
    input_signature=[tf.TensorSpec(shape=[None, X_train.shape[1]], dtype=tf.float64)],
)

save_dir = "/home/smmrrr/TEM_Analysis/TEM_Analysis/models/"
save_name = "test_gp"
tf.saved_model.save(model, save_dir+save_name)


In [ ]:
Y_train_sub[var+'_pred'] = xgb_model.predict(X_train_sub)
Y_test[var+'_pred'] = xgb_model.predict(X_test)

r2_train = r2_score(Y_train_sub[var], Y_train_sub[var+'_pred'])
r2_test = r2_score(Y_test[var], Y_test[var+'_pred'])
rmse_train = np.sqrt(mean_squared_error(Y_train_sub[var], Y_train_sub[var+'_pred']))
rmse_test = np.sqrt(mean_squared_error(Y_test[var], Y_test[var+'_pred']))
bias_train = np.mean(Y_train_sub[var+'_pred'] - Y_train_sub[var])
bias_test = np.mean(Y_test[var+'_pred'] - Y_test[var])

